# Loading the libraries

| Platform |
|----------|
| [![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/osllmai/inDox/blob/master/cookbook/indoxGen/generated_with_llm_judge.ipynb) |
| [![GitHub](https://img.shields.io/badge/GitHub-Repository-blue?logo=github)](https://github.com/osllmai/inDox/blob/master/cookbook/indoxGen/generated_with_llm_judge.ipynb) |


In [ ]:
# %pip install  indoxGen openai tenacity dotenv loguru  tenacity

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
NVIDIA_API_KEY = os.environ['NVIDIA_API_KEY']

### Example Data:

This cell defines two sets of columns and corresponding example data for generating synthetic data, one focused on general demographic information and the other on medical data.

1. **General Data**:
   - `columns`: A list of three column names: `"name"`, `"age"`, and `"occupation"`. These represent typical demographic information fields.
   - `example_data`: A list of dictionaries where each dictionary represents a person's demographic data. Two individuals are included:
     - **Alice Johnson** (35 years old, Manager).
     - **Bob Williams** (42 years old, Accountant).

2. **Medical Data**:
   - `columns_medical`: A list of six column names related to medical records, including `"Patient ID"`, `"Patient Name"`, `"Diagnosis Code"`, `"Procedure Code"`, `"Total Charge"`, and `"Insurance Claim Amount"`.
   - `examples_medical`: A list of dictionaries containing medical examples in string format. Each dictionary provides a description of a patient's visit, including the patient ID, name, diagnosis code (e.g., ICD-10), procedure code (e.g., CPT), total charge, and the insurance claim amount.
     - **Example 1**: Patient John Doe with diagnosis code J20.9 (acute bronchitis) and procedure 99203 (office visit).
     - **Example 2**: Patient Johnson Smith with diagnosis code M54.5 (low back pain) and procedure 99213 (office visit).
     - **Example 3**: Patient Emily Stone with diagnosis code E11.9 (type 2 diabetes) and procedure 99214 (office visit).

This setup lays the foundation for generating two distinct types of synthetic datasets: one focusing on demographic data and the other on medical records. Each data type will likely follow different generation methods.


In [2]:
columns = ["name", "age", "occupation"]
example_data = [
    {"name": "Alice Johnson", "age": 35, "occupation": "Manager"},
    {"name": "Bob Williams", "age": 42, "occupation": "Accountant"}
]

columns_medical = ["Patient ID","Patient Name","Diagnosis Code","Procedure Code","Total Charge","Insurance Claim Amount"]
examples_medical = [
    {
        "example": """Patient ID: 123456, Patient Name: John Doe, Diagnosis Code: 
        J20.9, Procedure Code: 99203, Total Charge: $500, Insurance Claim Amount: $350"""
    },
    {
        "example": """Patient ID: 789012, Patient Name: Johnson Smith, Diagnosis 
        Code: M54.5, Procedure Code: 99213, Total Charge: $150, Insurance Claim Amount: $120"""
    },
    {
        "example": """Patient ID: 345678, Patient Name: Emily Stone, Diagnosis Code: 
        E11.9, Procedure Code: 99214, Total Charge: $300, Insurance Claim Amount: $250"""
    },
]

### Setting Up API Clients for Synthetic Data Generation

This cell establishes connections to two API clients, which are used to interact with large language models (LLMs) for synthetic data generation tasks.

1. **Imports**:
   - `from indoxGen.llms import OpenAi`: Imports the `OpenAi` class from the `indoxGen.llms` module, enabling access to the OpenAI API for generating synthetic data.

2. **Initialization of API Clients**:
   - **OpenAI Client**:
     ```python
     openai = OpenAi(api_key=OPENAI_API_KEY, model="gpt-4o-mini")
     ```
     Initializes an `OpenAi` client using the provided `OPENAI_API_KEY` to connect to the `"gpt-4o-mini"` model. This client will be used for generating synthetic data based on the GPT-4 mini variant, providing more lightweight and efficient data generation.

   - **Nemotron (NVIDIA) Client**:
     ```python
     nemotron = OpenAi(api_key=NVIDIA_API_KEY, model="nvidia/nemotron-4-340b-instruct", base_url="https://integrate.api.nvidia.com/v1")
     ```
     Initializes another `OpenAi` client, this time for interacting with the `"nvidia/nemotron-4-340b-instruct"` model, using the `NVIDIA_API_KEY` and connecting to NVIDIA's integration API via the `base_url`. The Nemotron model is a high-capacity language model designed for complex data generation tasks.

This setup allows for generating synthetic data using two different models: the lightweight GPT-4 mini for more general tasks and the NVIDIA Nemotron model for heavier, more complex data generation.


In [3]:
from indoxGen.llms import OpenAi
import pandas as pd

openai = OpenAi(api_key=OPENAI_API_KEY,model="gpt-4o-mini")

nemotron = OpenAi(api_key=NVIDIA_API_KEY, model="nvidia/nemotron-4-340b-instruct",
                  base_url="https://integrate.api.nvidia.com/v1")

### Initializing Synthetic Data Generator

This cell sets up the synthetic data generation pipeline by initializing an instance of `SyntheticDataGenerator`. The generator leverages two language models (LLMs) to create realistic synthetic data based on the provided examples and user instructions.

1. **Imports**:
   - `from indoxGen.synthCore import SyntheticDataGenerator`: Imports the `SyntheticDataGenerator` class from the `indoxGen.synthCore` module, which is responsible for generating synthetic data based on specified parameters.

2. **Initializing the `SyntheticDataGenerator`**:
    The `SyntheticDataGenerator` is initialized with the following parameters:
- **generator_llm**: The main language model (`nemotron`) used for generating the synthetic data.
- **judge_llm**: A secondary language model (`openai`) used for evaluating the generated data, ensuring its quality and accuracy.
- **columns**: Specifies the structure of the synthetic data, including `"name"`, `"age"`, and `"occupation"`.
- **example_data**: Provides example entries to guide the generation process.
- **user_instruction**: A detailed instruction for generating synthetic data, ensuring diversity in names, ages, occupations, and race. It also ensures that the generated data covers both common and rare procedures, along with appropriate age ranges.
- **verbose**: Controls the verbosity of the generator, with `1` enabling detailed output during the generation process.


In [4]:
from indoxGen.synthCore import GenerativeDataSynth
generator = GenerativeDataSynth(
    generator_llm=openai,
    columns=columns,
    example_data=example_data,
    user_instruction="Generate realistic data including name, age and occupation. Ensure a mix of common and rare procedures, varying race, and appropriate date ranges for age.",
    verbose=1
)

In [5]:
medical_billing_generator = GenerativeDataSynth(
    generator_llm=nemotron,
    judge_llm=openai,
    columns=columns_medical,
    example_data=examples_medical,
    user_instruction="Generate realistic medical billing data including patient IDs, Patient Name, diagnosis codes, Total Charge, and Insurance Claim Amount. Ensure a mix of common and rare procedures, varying charge amounts, and appropriate date ranges for a typical healthcare provider.",
    verbose=1
)

### Generating Synthetic Data

This cell generates synthetic data based on the configuration of the `SyntheticDataGenerator` instance.

- **generated_data**: The result of calling the `generate_data` method on the `generator` instance. It stores the synthetic data created by the language model.
  
- **num_samples=20**: Specifies that the generator should create 20 samples of synthetic data. This controls the size of the output dataset.

The generated data will follow the structure and content outlined in the `columns`, `example_data`, and `user_instruction` provided earlier, with the `generator_llm` producing the data and the `judge_llm` validating it for quality and accuracy.


In [6]:
# Generate data
generated_data = generator.generate_data(num_samples=20)

INFO: Generated data point: {'name': 'Rajesh Patel', 'age': '39', 'occupation': 'Software Engineer'}
INFO: Generated data point: {'name': 'Fatima Al-Mansoori', 'age': '37', 'occupation': 'Data Scientist'}
INFO: Generated data point: {'name': 'Chloe Thompson', 'age': '41', 'occupation': 'Graphic Designer'}
INFO: Generated data point: {'name': 'Marcus Chen', 'age': '36', 'occupation': 'Marketing Specialist'}
INFO: Generated data point: {'name': 'Jamal Robinson', 'age': '39', 'occupation': 'Civil Engineer'}
INFO: Generated data point: {'name': 'Fatima Al-Mansoori', 'age': '37', 'occupation': 'Data Scientist'}
INFO: Generated data point: {'name': 'Sofia Patel', 'age': '41', 'occupation': 'Healthcare Administrator'}
INFO: Generated data point: {'name': 'Marcus Chen', 'age': '36', 'occupation': 'Marketing Specialist'}
INFO: Generated data point: {'name': 'Jamal Thompson', 'age': '39', 'occupation': 'Civil Engineer'}
INFO: Generated data point: {'name': 'Lila Rodriguez', 'age': '37', 'occupat

In [7]:
generated_data

,name,age,occupation
0,Rajesh Patel,39,Software Engineer
1,Fatima Al-Mansoori,37,Data Scientist
2,Chloe Thompson,41,Graphic Designer
3,Marcus Chen,36,Marketing Specialist
4,Jamal Robinson,39,Civil Engineer
5,Fatima Al-Mansoori,37,Data Scientist
6,Sofia Patel,41,Healthcare Administrator
7,Marcus Chen,36,Marketing Specialist
8,Jamal Thompson,39,Civil Engineer
9,Lila Rodriguez,37,Data Scientist


In [8]:
medical_billing_data = medical_billing_generator.generate_data(num_samples=6)

INFO: Generated data point: {'Patient ID': '987654', 'Patient Name': 'Olivia Green', 'Diagnosis Code': 'I10', 'Procedure Code': '36415', 'Total Charge': '$2,500', 'Insurance Claim Amount': '$2,000'}
INFO: Generated data point: {'Patient ID': '654321', 'Patient Name': 'Sophia Brown', 'Diagnosis Code': 'G47.33', 'Procedure Code': '92507', 'Total Charge': '$1,800', 'Insurance Claim Amount': '$1,500'}
INFO: Generated data point: {'Patient ID': '1234567', 'Patient Name': 'Michael Johnson', 'Diagnosis Code': 'K21.9', 'Procedure Code': '43235', 'Total Charge': '$3,200', 'Insurance Claim Amount': '$2,800'}
INFO: Generated data point: {'Patient ID': '8765432', 'Patient Name': 'Ava Williams', 'Diagnosis Code': 'F32.9', 'Procedure Code': '90837', 'Total Charge': '$1,200', 'Insurance Claim Amount': '$900'}
INFO: Generated data point: {'Patient ID': '9876543', 'Patient Name': 'Olivia Davis', 'Diagnosis Code': 'I10', 'Procedure Code': '36.12', 'Total Charge': '$2,500', 'Insurance Claim Amount': '$2,

In [9]:
medical_billing_data

,Patient ID,Patient Name,Diagnosis Code,Procedure Code,Total Charge,Insurance Claim Amount
0,987654,Olivia Green,I10,36415,"$2,500","$2,000"
1,654321,Sophia Brown,G47.33,92507,"$1,800","$1,500"
2,1234567,Michael Johnson,K21.9,43235,"$3,200","$2,800"
3,8765432,Ava Williams,F32.9,90837,"$1,200",$900
4,9876543,Olivia Davis,I10,36.12,"$2,500","$2,000"
5,6549873,Sophia Brown,G47.33,95810,"$1,800","$1,500"


In [10]:
print("Generated Data is done!")

Generated Data is done!
